load the respective training data  
load the respective test data  
add function(s) specific to that dataset  
train each model on the training data  
pickle the trained model (for safe keeping, not totally necessary)  
make predictions on the test data  
pickle the results  

In [1]:
import os

import pandas as pd
import numpy as np

from predict_submit_funcs import submit_fit_score_pred_log, submit_fit_score_pred_G_NB, submit_fit_score_pred_M_NB, submit_fit_score_pred_rfc

import pickle


In [2]:
os.chdir('../Data/')

In [3]:
ord_df = pd.read_csv('orders.csv')
final_order_id = ord_df[ord_df.eval_set=='test'].loc[:,['order_id','user_id']]

# Without adding department

In [4]:
train_df = pd.read_pickle('full_features_train')
test_df = pd.read_pickle('full_features_test')

## Logistic Regression

In [5]:
predictions = submit_fit_score_pred_log(train_df, test_df, final_order_id,C=11250)
pd.DataFrame.to_csv(predictions,'predictions_full_features_log')

## Gaussian Naive Bayes

In [6]:
predictions = submit_fit_score_pred_G_NB(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_g_nb')

## Random Forest

In [7]:
predictions = submit_fit_score_pred_rfc(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_rfc')

# After Scaling Data

In [8]:
train_df = pd.read_pickle('full_features_scaled_train')
test_df = pd.read_pickle('full_features_scaled_test')

## Logistic Regression

In [9]:
predictions = submit_fit_score_pred_log(train_df, test_df, final_order_id,C=10611)
pd.DataFrame.to_csv(predictions,'predictions_full_features_scaled_log')

# With Polynomial Features

In [11]:
train_df = pd.read_pickle('full_features_poly_train')
test_df = pd.read_pickle('full_features_poly_test')

## Gaussian Naive Bayes

In [12]:
predictions = submit_fit_score_pred_G_NB(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_poly_g_nb')

# Scaled Polynomial Features

In [4]:
train_df = pd.read_pickle('full_features_scaled_poly_train')
test_df = pd.read_pickle('full_features_scaled_poly_test')

## Logistic Regression

In [ ]:
predictions, lr = submit_fit_score_pred_log(train_df, test_df, final_order_id,C=11250)
pd.DataFrame.to_csv(predictions,'predictions_full_features_scaled_poly_log')

with open("../Models/full_features_scaled_poly_log_model.pkl", "wb") as f:
    pickle.dump(lr, f)

## Gaussian Naive Bayes

In [5]:
predictions, gnb = submit_fit_score_pred_G_NB(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_scaled_poly_g_nb')

with open("../Models/full_features_scaled_poly_log_g_nb.pkl", "wb") as f:
    pickle.dump(gnb, f)

# Adding Departments

In [4]:
train_df = pd.read_pickle('full_features_with_dep_train')
test_df = pd.read_pickle('full_features_with_dep_test')

## Gaussian Naive Bayes

In [6]:
predictions, gnb = submit_fit_score_pred_G_NB(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_dep_g_nb')

with open("../Models/full_features_with_dep_g_nb.pkl", "wb") as f:
    pickle.dump(gnb, f)

## Multinomial Naive Bayes

In [5]:
predictions, mnb = submit_fit_score_pred_M_NB(train_df, test_df, final_order_id)
pd.DataFrame.to_csv(predictions,'predictions_full_features_dep_m_nb')

with open("../Models/full_features_with_dep_m_nb.pkl", "wb") as f:
    pickle.dump(mnb, f)

# Combining Predictions

``` python 
pd.read_csv('predictions_full_features_log')
pd.read_csv('predictions_full_features_g_nb')
pd.read_csv('predictions_full_features_rfc')
pd.read_csv('predictions_full_features_scaled_log')
pd.read_csv('predictions_full_features_poly_g_nb')
pd.read_csv('predictions_full_features_scaled_poly_log')
pd.read_csv('predictions_full_features_scaled_poly_g_nb')
pd.read_csv('predictions_full_features_dep_g_nb')
pd.read_csv('predictions_full_features_dep_m_nb')
```

In [89]:
csvs = ['predictions_full_features_g_nb',
'predictions_full_features_rfc','predictions_full_features_scaled_log',
'predictions_full_features_scaled_poly_log',
'predictions_full_features_scaled_poly_g_nb','predictions_full_features_dep_g_nb']

df = pd.read_csv('predictions_full_features_dep_m_nb',index_col='Unnamed: 0')
for csv in csvs:
    df.prediction = pd.read_csv(csv).prediction + df.prediction

df.prediction = df.prediction / 7



In [90]:
df.head()

,product_id,user_id,prediction,order_id
0,1,764,0.714286,2563471
1,40199,764,0.142857,2563471
2,8859,764,1.000000,2563471
3,22340,764,0.857143,2563471
4,48667,764,0.142857,2563471


In [83]:
conditions = [
    (df.prediction > 0.4),
    (df.prediction < 0.4)
]
choices = [1,0]
df.prediction = np.select(conditions, choices)

In [84]:
def format_for_submission(predictions):
    predictions['product_id'] = predictions.prediction * predictions.product_id
    predictions.drop('prediction',axis=1,inplace=True)
    predictions = (pd.DataFrame(predictions.groupby('order_id')
                    ['product_id'].apply(products_concat)).reset_index())
    predictions.columns = ['order_id','products']
    return predictions

In [85]:
def products_concat(series):
    out = ''
    for product in series:
        if product > 0:
            out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [86]:
predictions = format_for_submission(df)

In [87]:
df.head()

,product_id,user_id,order_id
0,1,764,2563471
1,0,764,2563471
2,8859,764,2563471
3,22340,764,2563471
4,0,764,2563471


In [88]:
pd.DataFrame.to_csv(predictions,'predictions_ensemble.csv',index=False)